<center><h2>ALTeGraD 2023<br>Lab Session 3: Transfer learning for NLP</h2> 24 / 10 / 2023<br> Dr. G. Shang, H. Abdine<br><br>


<b>Student name:</b> Balthazar Neveu

</center>

<br><br>
In this lab we will:
* Implement and pretrain a language model with transformer architecture.
* Use the pretrained model (transfer learning) to perform a sentiment analysis task which consists of classifying some books reviews into positive and negative ones.
* Compare the performance of the pretrained model to a model trained from scratch.
 <br>

<b>The deadline for this lab is October 31, 2023 11:59 PM.</b> More details about the submission and the architecture for this lab can be found in the handout PDF.

In [ ]:
!wget https://raw.githubusercontent.com/moussaKam/transfer_learning_transformers/main/pretraining_subset.txt
!wget https://raw.githubusercontent.com/moussaKam/transfer_learning_transformers/main/dict.txt
!wget https://raw.githubusercontent.com/moussaKam/transfer_learning_transformers/main/pretrained_model_4layers.pt
!wget https://raw.githubusercontent.com/moussaKam/transfer_learning_transformers/main/cls-books/train.review.spm
!wget https://raw.githubusercontent.com/moussaKam/transfer_learning_transformers/main/cls-books/train.label
!wget https://raw.githubusercontent.com/moussaKam/transfer_learning_transformers/main/cls-books/test.review.spm
!wget https://raw.githubusercontent.com/moussaKam/transfer_learning_transformers/main/cls-books/test.label
!wget https://raw.githubusercontent.com/moussaKam/transfer_learning_transformers/main/sentencepiece.french.model
!head -5 dict.txt

!pip install prettytable
!pip install sympy

In [1]:
import math

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from pathlib import Path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

/home/bneveu/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:126: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [2]:
path_vocab = Path("dict.txt")
assert path_vocab.exists()
pretraining_path_data_train = Path("pretraining_subset.txt")
assert pretraining_path_data_train.exists()

downstream_path_data_train = Path("train.review.spm")
assert downstream_path_data_train.exists()
downstream_path_labels_train = Path("train.label")
assert downstream_path_labels_train.exists()
downstream_path_data_valid = Path("test.review.spm")
assert downstream_path_data_valid.exists()
downstream_path_labels_valid = Path("test.label")
assert downstream_path_labels_valid.exists()

In [3]:
pretrained_model = Path("pretrained_model_4layers.pt")
assert pretrained_model.exists()

# Tensor convention for NLP
`[L, N, D]`
- L sequence length
- N batch size
- V vocabulary dimension `ntokens`
- E embeddings dimension `embedding_dim`
- D hidden dimension

### Simplification:
- `E=D` hidden dimension set equal to th embedding dimension for simplicity in the following code `nhid = embedding_dim`



### The Model

In [4]:
class PositionalEncoding(nn.Module):
    """Add fixed precomputed positional encoding to the embeddings
    Add means (=literally addition)
    """
    def __init__(self, embdeddings_dim: int , dropout: float =0.1, max_len: int =5000):
        """Precompute a positional encoding vector of length `max_len`

        Args:
            embdeddings_dim (int): dimension of word embeddings. Note th
            dropout (float, optional): dropout ratio. Defaults to 0.1.
            max_len (int, optional): maximum sequence length. Defaults to 5000.
        """
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, embdeddings_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, embdeddings_dim, 2).float() * (-math.log(10000.0) / embdeddings_dim)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pe", pe)

    def forward(self, x: torch.FloatTensor) -> torch.FloatTensor:
        """Add positional encoding to the word embeddings.
        Simply add the pre

        Args:
            x (torch.FloatTensor): embeddings tensor [L, N, D]

        Returns:
            torch.FloatTensor: Enhanced embeddings tensor, ready to go straight to the transformer blocks. 
        """
        x = x + self.pe[: x.size(0), :]
        return self.dropout(x)

In [5]:
class TransformerModel(nn.Module):
    """Transformer base model 
    =========================
    - embedding from word to vectors
    - add positional encoding
    - `nlayers` * transformer blocks
    """
    def __init__(self, ntokens:int, nhead:int, nhid:int, nlayers:int, dropout=0.5):
        """Transformer base model

        Args:
            ntokens (int): the size of vocabulary
            nhead (int): number of heads in each of the MHA models
            nhid (int): hidden dimension of the model. assume `embedding_dim` = `nhid`
            nlayers (int): number of nn.TransformerEncoderLayer in nn.TransformerEncoder
            dropout (float, optional): dropout value. Defaults to 0.5.
        """
        super(TransformerModel, self).__init__()
        self.model_type = "Transformer"
        embedding_dim = nhid # use the same embedding & hidden dimensions
        self.encoder = nn.Embedding(ntokens, embedding_dim) # fill me, nhid = the dim_embed
        self.pos_encoder = PositionalEncoding(nhid, dropout=dropout) #fill me, the PositionalEncoding class is implemented in the next cell
        
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=nhid, # input dimension to the transformer encoder layer
            nhead=nhead, # number of heads for MHA (Multi-head attention)
            dim_feedforward=nhid, # output dimension of the MLP on top of the transformer.
            dropout=dropout
        ) # we assume nhid = d_model = dim_feedforward
        
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layers,
            num_layers=nlayers
        )
        self.nhid = nhid
        self.init_weights()
    
    @staticmethod
    def generate_square_subsequent_mask(sz: int) -> torch.FloatTensor:
        """Generate causality mask = mask future tokens for next word prediction

        Args:
            sz (int): mask size M

        Returns:
            torch.FloatTensor: squares matrix [M, M] to mask the attention matrix.
        """
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = (
            mask.float()
            .masked_fill(mask == 0, float("-inf"))
            .masked_fill(mask == 1, float(0.0))
        )
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)

    def forward(
            self, src: torch.LongTensor,
            src_mask: torch.FloatTensor
        ) -> torch.FloatTensor:
        """Embdeddings, positional encoders, go trough `nlayers` of residual {multi (`nhead`) attention heads + MLP}.

        Args:
            src (torch.LongTensor): [L, N, V] sequence of tokens , V=vocabu
            src_mask (torch.FloatTensor): [L, L] squared mask

        Returns:
            torch.FloatTensor: encoded sequence [L, N, D]
        """
        src = self.encoder(src) * math.sqrt(self.nhid) #embed [L, N, V] -> [L, N, E]
        src = self.pos_encoder(src) # [L, N, E]  - add positional encoding
        output = self.transformer_encoder(src, mask=src_mask)
        return output

In [6]:
class ClassificationHead(nn.Module):
    def __init__(self, nhid: int, nclasses: int):
        """Linear classification head -> returns logits (not probabilities)

        Args:
            nhid (int): hidden dimension
            nclasses (int): number of classes.
        """
        super(ClassificationHead, self).__init__()
        self.decoder = nn.Linear(nhid, nclasses)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: torch.FloatTensor) -> torch.FloatTensor:
        """Classify encoded feature vectors

        Args:
            src (torch.FloatTensor): Encoded feature vectors [L, N, D]

        Returns:
            torch.FloatTensor: Logits (no softmax applied)
        """
        output = self.decoder(src)
        return output

class Model(nn.Module):
    def __init__(self, ntoken: int, nhead: int, nhid: int, nlayers: int, nclasses: int, dropout: float=0.5):
        """TransformerModel+ClassificationHead
        
        This allows defining a model for next word prediction (classification with ntoken classes)
        Or other downstream tasks if the base `TransformerModel` is pretrained

        Args:
        
            ntoken (int): size of vocabulary for (`TransformerModel`)
            nhead (int): number of heads in each of the MHA models (`TransformerModel`)
            nhid (int): hidden dimension of the model. assume `embedding_dim` = `nhid`
            nlayers (int):  number of nn.TransformerEncoderLayer in nn.TransformerEncoder (`TransformerModel`)
            nclasses (int): number of output classes in the classifier `ClassificationHead`
                - =size of vocabulary for next word prediction
                - other for downstream tasks like sentiment analyzis.
            dropout (float, optional): _description_. Defaults to 0.5.  (`TransformerModel`)
        """
        super(Model, self).__init__()
        self.base = TransformerModel(ntoken, nhead, nhid, nlayers, dropout=dropout)
        self.classifier = ClassificationHead(nhid, nclasses)

    def forward(self, src:torch.LongTensor, src_mask: torch.FloatTensor) -> torch.FloatTensor:
        """Encoder + linear classifier

        Args:
            src (torch.LongTensor): sequence of tokens [L, N, V]
            src_mask (torch.FloatTensor): [L, L] squared mask.

        Returns:
            torch.FloatTensor: [N, C]
        """
        # base model
        x = self.base(src, src_mask)
        # classifier model
        output = self.classifier(x)
        return output

### Causal attention mask & useless computations (question 1)

In [7]:
sentence_length_test = 5
src_mask = TransformerModel.generate_square_subsequent_mask(sentence_length_test)
useless_computations = sentence_length_test*(sentence_length_test-1)//2
assert int( ((-src_mask).isinf()).sum()) == useless_computations
print(f"{useless_computations} useless computations for a sequence of {sentence_length_test} tokens")
src_mask

10 useless computations for a sequence of 5 tokens


tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])

# Unit test

In [8]:
def test_transformer_based_classifier():
    ntokens = 100 #  V the size of vocabulary
    nhid = 200  # hidden dimension
    nlayers = 4  # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
    nhead = 2  # the number of heads in the multiheadattention models
    dropout = 0  # the dropout value
    nclasses = ntokens # classification to get output words in the same language
    model = Model(ntokens, nhead, nhid, nlayers, nclasses, dropout).to(device)
    dummy_input = torch.tensor([[2, 6, 2, 5, 43, 21], [8, 5, 3, 42, 43, 21]]).to(device)

    sequence_length = dummy_input.shape[0] #L
    batch_size = dummy_input.shape[1] #N

    src_mask = TransformerModel.generate_square_subsequent_mask(sequence_length).to(device)
    assert list(src_mask.shape) == [sequence_length,sequence_length]
    # batch dimension N is not involved in the mask computation! We assume all sequences in the batch has the same sequence length L
    out = model.forward(dummy_input, src_mask)
    expected_size = [sequence_length, batch_size, nclasses]
    assert list(out.shape) == expected_size, f"{out.shape}, {expected_size}"
    print(out.shape)
test_transformer_based_classifier()

torch.Size([2, 6, 100])


/home/bneveu/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:255: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because  encoder_layer.self_attn.batch_first was not True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


## Vocabulary and Tokenization

In [9]:
SRC = "source_sequence"
TGT = "target"
SOS = "<sos>"
PAD = "<pad>"
EOS = "<eos>"
OOV = "<oov>"
LM_TASK = "language_modeling"
DS_TASK = "classification"
token2ind = {SOS: 0, PAD : 1, EOS: 2, OOV: 3} # the 4 first indices are reserved to special tokens
offset = max(token2ind.values())+1
with open(path_vocab, "r") as f:
    for idx, line in enumerate(f):
        word = line.split()[0].strip()
        token2ind[word] = idx+offset
ind2token = {index: token for token, index in token2ind.items()}
print(ind2token[1111], ind2token[0])

▁trop <sos>


### Data Loader


In [10]:
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from pathlib import Path
from typing import Dict, List, Tuple

class Dataset(Dataset):
    def __init__(
        self,
        path_documents: Path,
        path_labels: Path = None,
        token2ind: Dict[str, int]={},
        max_len: int=512,
        task: str=LM_TASK,
    ):
        self.task = task
        self.max_len = max_len
        self.token2ind = token2ind
        self.documents = []
        self.labels = []
        with open(path_documents, "r") as f1:
            for line in f1:
                self.documents.append(line.strip())
        if task == "classification":
            with open(path_labels, "r") as f1:
                for line in f1:
                    self.labels.append(int(line.strip()))
            assert len(self.labels) == len(self.documents)
        self.oov_index = self.token2ind[OOV]
    def __len__(self):
        return len(self.documents)

    def __getitem__(self, index: int):
        sequence = self.documents[index].split()
        if len(sequence) > self.max_len - 1:
            sequence = sequence[: self.max_len - 1] 
        
        source_sequence = [self.token2ind.get(token, self.oov_index) for token in sequence]
        source_sequence.insert(0, self.token2ind[SOS]) 
        # (constract the input sequence using token2ind, sequence and special tokens)
        if self.task == LM_TASK:
            target = source_sequence[1:] # offset the sequence by one
            # A, B , C, D , <EOS>
            target.append(self.token2ind[EOS])
            assert len(target) == len(source_sequence)
        elif self.task == DS_TASK:
            target = [self.labels[index]]
        sample = {
            SRC: torch.tensor(source_sequence),
            TGT: torch.tensor(target),
        }
        return sample


def collate_sentences_keep_dim(batch: List[Dict[str, torch.LongTensor]]) -> Tuple[torch.LongTensor, torch.LongTensor]:
    """Uniformize batches (have the same sentence length with padding for all sentences across the batch)

    Args:
        batch (List[Dict[str, torch.LongTensor]]): List of dict samples containing 

    Returns:
        Tuple[torch.LongTensor, torch.LongTensor]: 
            - source [L, N, V]
            where L is the maximum length along all sentences in the batch
            - target 
                - [L, N, V] for language modeling task
                - [N, C] for classification with C the number of classes
            
    """
    source_sequences = pad_sequence(
        #we use padding to match the length of the sequences in the same batch
        [sample[SRC] for sample in batch], padding_value=token2ind[PAD]
    )
    target = pad_sequence(
        [sample[TGT] for sample in batch], padding_value=token2ind[PAD]
    )
    return source_sequences, target


def collate_sentences(batch: List[Dict[str, torch.LongTensor]]) -> Tuple[torch.LongTensor, torch.LongTensor]:
    source_sequences, target = collate_sentences_keep_dim(batch)
    return source_sequences, target.reshape(-1)

def get_loader(
    path_documents :Path,
    path_labels: Path = None,
    token2ind : Dict[str, int]={},
    max_len: int =512,
    batch_size: int = 32,
    task: str=LM_TASK,
    collate_fn = collate_sentences,
    shuffle=True
):
    dataset = Dataset(
        path_documents,
        path_labels=path_labels,
        token2ind=token2ind,
        max_len=max_len,
        task=task,
    )
    data_loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=collate_fn,
        pin_memory=True,
        drop_last=True,
    )
    return data_loader

## Sanity check on data loader and collate functions

In [11]:
N = 32
data_loader = get_loader(
    pretraining_path_data_train,
    token2ind=token2ind,
    batch_size=N,
    task=LM_TASK,
    collate_fn=collate_sentences_keep_dim
)
token2ind[OOV]
it = iter(data_loader)

def tensor_to_sentence(tensor_sentence):
        return " ".join([ind2token.get(el, "not found").replace("▁", "") for el in tensor_sentence.numpy() if el != token2ind[PAD]])

for u in range(5):
    sampled_batch = next(it)
    # first[0].shape, first[1].shape
    # first[0][:, 0] # first sentence # L, N
    first_sentence = sampled_batch[0][:, 0]
    def tensor_to_sentence(tensor_sentence):
        return " ".join([ind2token.get(el, "not found").replace("▁", "") for el in tensor_sentence.numpy() if el != token2ind[PAD]])
    print(tensor_to_sentence(first_sentence))
    print(tensor_to_sentence(sampled_batch[1][:, 0]))

<sos> A Population Base d , Pers on Spe ci fic Analy sis of Pat tern s of U se by Seniors Numéro du projet : 66 10 -2 11 7 -60 2 D eter min ants of Seniors ' Inde pend ence :
A Population Base d , Pers on Spe ci fic Analy sis of Pat tern s of U se by Seniors Numéro du projet : 66 10 -2 11 7 -60 2 D eter min ants of Seniors ' Inde pend ence : <eos>
<sos> En ligne de mire : les derniers ajustements quant à la fiscalité écologique dans le budget 2014 ; et la grande loi annoncée sur la transition énergétique .
En ligne de mire : les derniers ajustements quant à la fiscalité écologique dans le budget 2014 ; et la grande loi annoncée sur la transition énergétique . <eos>
<sos> Dans le scénario " diversité ", il est considéré que la réduction des consommations d ' énergie ne pourra être poussée jusqu ' à 50 %.
Dans le scénario " diversité ", il est considéré que la réduction des consommations d ' énergie ne pourra être poussée jusqu ' à 50 %. <eos>
<sos> Mais , il y a un la tif réduit où se c

# Model definition

In [78]:
import sympy
from prettytable import PrettyTable

def count_parameters(my_model, table_flag=True, details=[]):
    table = PrettyTable(["Modules", "Parameters", "Analytic validation", "Analytic formula"])
    total_params = 0
    for index, (name, parameter) in enumerate(my_model.named_parameters()):
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        detail = ["", ""] if index>=len(details) else details[index]
        table.add_row([name, params, detail[0], detail[1]])
        total_params += params
    if table_flag:
        print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [118]:
ntokens = len(ind2token) # the size of vocabulary
print(len(ind2token))
nhid = 200  # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 4  # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # the number of heads in the multiheadattention models
dropout = 0  # the dropout value

nclasses = 2 # for classification task only

model_pretraining = Model(ntokens, nhead, nhid, nlayers, ntokens, dropout).to(device)
count_parameters(model_pretraining, table_flag=False)

# model_pretraining = Model(ntokens, 4*nhead, nhid, nlayers, ntokens, dropout).to(device)
# count_parameters(model_pretraining, table=False)
# 21018401 nhead= 4
# 21018401 nhead = 2

50001
Total Trainable Params: 21018401


/home/bneveu/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:255: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because  encoder_layer.self_attn.batch_first was not True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


21018401

# Question 3: Parameters amount analytic validation

In [128]:
from sympy import symbols, Eq, solve, sympify
from IPython.display import display, Markdown

V, D, A, C, M = symbols("V, D, A, C, M") #(nhid*ntokens)
embedding_size = V*D
true_values = {
    M: int(nlayers),
    V: int(ntokens),
    D: int(nhid),
    A: int(nhead),
    C: int(ntokens)
}
list_computations = []
def get_result(name, expr_:str):
    expr = sympify(expr_)
    res = expr.evalf(subs=true_values)
    equation = Eq(expr, int(res))
    latex_eq = sympy.latex(equation)
    eq_original  = sympy.latex(sympify(expr_, evaluate=False))
    display(Markdown(f"{name}: ${eq_original}$ =  $ {latex_eq} $"))
    list_computations.append([int(res) , str(expr).replace("**2", "²")]) #[f"{name}: ${eq_original}$ =  $ {latex_eq} $", ])
get_result("Embeddings linear projection weights", "V*D")
get_result("MHA inputs Weights (Key, Query, Value)", "A*3*(D*(D/A))")
get_result("MHA inputs Bias (Key, Query, Value)", "A*3*(D/A)")

get_result("MHA output Weights", "A*((D/A)*D)")
get_result("MHA output Bias", "A*(D/A)")

for linear_layer in range(1, 2+1):
    get_result(f"Position-wise #{linear_layer} Linear MLP weights", "D*D")
    get_result(f"Position-wise #{linear_layer} Linear MLP bias", "D")

for linear_layer in range(1, 2+1):
    get_result(f"Layer norm #{linear_layer} coefficient", "D")
    get_result(f"Layer norm #{linear_layer} bias", "D")


get_result("Classifier weights", "D*C")
get_result("Classifier bias", "C")

# Layer Normalization applies per-element scale and bias... so to each D dimension
model_dissect = Model(ntokens, nhead, nhid, 1, ntokens, dropout).to(device)
count_parameters(model_dissect, table_flag=True, details=list_computations)

print(20*"⚠️ " + "Simplified M=1 layer model")

Embeddings linear projection weights: $D V$ =  $ D V = 10000200 $

MHA inputs Weights (Key, Query, Value): $A 3 D D \frac{1}{A}$ =  $ 3 D^{2} = 120000 $

MHA inputs Bias (Key, Query, Value): $A 3 D \frac{1}{A}$ =  $ 3 D = 600 $

MHA output Weights: $\frac{A D D}{A}$ =  $ D^{2} = 40000 $

MHA output Bias: $\frac{A D}{A}$ =  $ D = 200 $

...

Layer norm #2 coefficient: $D$ =  $ D = 200 $

Layer norm #2 bias: $D$ =  $ D = 200 $

Classifier weights: $C D$ =  $ C D = 10000200 $

Classifier bias: $C$ =  $ C = 50001 $

+-------------------------------------------------------------+------------+---------------------+------------------+
|                           Modules                           | Parameters | Analytic validation | Analytic formula |
+-------------------------------------------------------------+------------+---------------------+------------------+
|                     base.encoder.weight                     |  10000200  |       10000200      |       D*V        |
|  base.transformer_encoder.layers.0.self_attn.in_proj_weight |   120000   |        120000       |       3*D²       |
|   base.transformer_encoder.layers.0.self_attn.in_proj_bias  |    600     |         600         |       3*D        |
| base.transformer_encoder.layers.0.self_attn.out_proj.weight |   40000    |        40000        |        D²        |
|  base.transformer_encoder.layers.0.self_attn.out_proj.bias  |    200     |         200         |        D         |
|       base.transformer_encoder.layers.0.linear1.weight

In [130]:

model_dissect = Model(ntokens, nhead, nhid, nlayers, ntokens, dropout).to(device)
total = count_parameters(model_dissect, table_flag=False, details=[])
get_result("Matches - Full model", "V*D + M*(6*(D^2+D) + 4*D) + C*(D+1)")


get_result("Classification model - frozen transformer backbone", "2*(D+1)")

Total Trainable Params: 21018401


Matches - Full model: $C \left(D + 1\right) + D V + M \left(4 D + 6 \left(D^{2} + D\right)\right)$ =  $ C \left(D + 1\right) + D V + M \left(6 D^{2} + 10 D\right) = 21018401 $

Classification model - frozen transformer backbone: $2 \left(D + 1\right)$ =  $ 2 D + 2 = 402 $

In [ ]:
# optimization parameters
criterion = nn.CrossEntropyLoss(ignore_index=token2ind['<pad>'])
lr = 0.0003  # learning rate
optimizer = torch.optim.Adam(model_pretraining.parameters(), lr=lr)

In [ ]:
def train(
    model,
    path_data_train: Path,
    path_labels_train: Path =None,
    path_data_valid: Path = None,
    save_interval: int =-1,
    log_interval: int=5,
    task: str=LM_TASK,
    batch_size: int =32,
    max_len=512,
    epoch=0,
    name=""
):
    model.train()
    total_loss = 0.0
    ntokens = len(token2ind)
    data_loader = get_loader(
        path_data_train,
        path_labels_train,
        token2ind,
        max_len=max_len,
        task=task,
        batch_size=batch_size,
    )

    losses = []
    for idx, data in enumerate(data_loader): # get a batch of samples
        # reset gradients to zero
        optimizer.zero_grad()
        
        src_mask = TransformerModel.generate_square_subsequent_mask(data[0].size(0)).to(
            device
        )
        input = data[0].to(device)
        # forward pass
        output = model(input, src_mask) 
        if task == DS_TASK:
            output = output[-1:, :]  # last vector only
        output = output.view(-1, output.shape[-1])
        target = data[1]
        target = target.to(device)
        loss =  criterion(output, target) # CROSS ENTROPY
        loss.backward() # compute gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5) # prevent exploding gradient
        optimizer.step()

        total_loss += loss.item()
        if idx % log_interval == 0 and idx > 0:
            cur_loss = total_loss / log_interval
            print(
                f"| epoch {epoch:3d} | {idx:5d}/{len(data_loader):5d} steps | "+
                f"loss {cur_loss:5.5f} | ppl {math.exp(cur_loss):8.3f}"
            )
            losses.append(cur_loss)
            total_loss = 0
    if epoch % save_interval == 0 and save_interval>=1:
        torch.save(model, f"weights_{task}{name}_{epoch:02d}.pt")
    return losses

In [ ]:
from_scratch_settings = [True, False]

from_scratch_valid_acc = []
pretrained_valid_acc = []
lr = 0.0001

for from_scratch in from_scratch_settings:
    model_ds = Model(ntokens, nhead, nhid, nlayers, 2, dropout).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model_ds.parameters(), lr=lr)
    if not from_scratch:
        print("=====PRETRAINED MODEL======")
        #load checkpoint
        checkpoint = torch.load("pretrained_model_4layers_no_class_head.pt")
        #load state dict
        model_ds.base.load_state_dict(checkpoint['model_state_dict'])
    else:
        print("=====Trainig FROM SCRATCH======")
    epochs = 15
    # @TODO : freeze transformer weights!
    # @TODO: check batch size subtlety on the last element (we want to classify the last feature of the sentence!)
    for epoch in range(1, epochs + 1):
        train(
            model_ds,
            downstream_path_data_train,
            downstream_path_labels_train,
            save_interval=-1,
            task=DS_TASK,
            # batch_size=8,
            batch_size=1, # TO AVOID THE NEED TO RETRIEVE THE RIGHT LAST TOKEN IN A BATCH
            log_interval=50,
            epoch=epoch,
            name="_from_scratch" if from_scratch else "_pretrained"
        )
        acc = evaluate_accuracy(
            data_loader_validation,
            model_ds
        )
        if from_scratch:
            from_scratch_valid_acc.append(acc)
        else:
            pretrained_valid_acc.append(acc)
        print(f"{from_scratch=} - VALIDATION ACCURACY {acc}")

In [ ]:

# "base.encoder.weight" = ntokens*nhid

In [ ]:

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
count_parameters(model_pretraining)

In [ ]:
# optimization parameters
criterion = nn.CrossEntropyLoss(ignore_index=token2ind['<pad>'])
lr = 0.0003  # learning rate
optimizer = torch.optim.Adam(model_pretraining.parameters(), lr=lr)

## Training loop

In [ ]:
def train(
    model: Model,
    path_data_train: Path,
    path_labels_train: Path =None,
    path_data_valid: Path = None,
    save_interval: int =-1,
    log_interval: int=5,
    task: str=LM_TASK,
    batch_size: int =32,
    max_len=512,
    epoch=0,
    name="",
    freeze_backbone=False
):
    model.train()
    if freeze_backbone:
        print("FREEZE TRANSFORMER BACKBONE")
        for param in model.base.parameters():
            param.requires_grad = False
    total_loss = 0.0
    ntokens = len(token2ind)
    data_loader = get_loader(
        path_data_train,
        path_labels_train,
        token2ind,
        max_len=max_len,
        task=task,
        batch_size=batch_size,
    )

    losses = []
    for idx, data in enumerate(data_loader): # get a batch of samples
        # reset gradients to zero
        optimizer.zero_grad()
        
        src_mask = TransformerModel.generate_square_subsequent_mask(data[0].size(0)).to(
            device
        )
        input = data[0].to(device)
        # forward pass
        output = model(input, src_mask) 
        if task == DS_TASK:
            output = output[-1:, :]  # last vector only
        output = output.view(-1, output.shape[-1])
        target = data[1]
        target = target.to(device)
        loss =  criterion(output, target) # CROSS ENTROPY
        loss.backward() # compute gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5) # prevent exploding gradient
        optimizer.step()

        total_loss += loss.item()
        if idx % log_interval == 0 and idx > 0:
            cur_loss = total_loss / log_interval
            print(
                f"| epoch {epoch:3d} | {idx:5d}/{len(data_loader):5d} steps | "+
                f"loss {cur_loss:5.5f} | ppl {math.exp(cur_loss):8.3f}"
            )
            losses.append(cur_loss)
            total_loss = 0
    if epoch % save_interval == 0 and save_interval>=1:
        torch.save(model, f"weights_{task}{name}_{epoch:02d}.pt")
    return losses

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()
# EMPTY THE GPU

In [ ]:
#pretraining on a tiny subset
import gc
gc.collect()
torch.cuda.empty_cache()

log_interval = 500
epochs = 2
for epoch in range(1, epochs + 1): #5
    train(
        pretraining_path_data_train,
        save_interval=-1,
        task=LM_TASK,
        batch_size=16,
        max_len=64,
        log_interval=log_interval,
        epoch=epoch
    )

## Text Generation


In [ ]:
model_pretraining = Model(ntokens, nhead, nhid, nlayers, ntokens).to(device)

#load the checkpoint
checkpoint = torch.load('pretrained_model_4layers.pt')
#load state dict
model_pretraining.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
import sentencepiece as spm

s = spm.SentencePieceProcessor(model_file='sentencepiece.french.model') #load sentencepiece model

#examples
encoded = s.encode_as_pieces("Bonjour les amis!")
decoded = s.decode_pieces(encoded)
print(encoded)
print(decoded)

In [ ]:
def infer_next_token(sent, model):
    model.eval()
    sent_pieces = s.encode_as_pieces(sent)
    source = [token2ind['<sos>']] + [token2ind[el] for el in sent_pieces] # list of tokens
    source = torch.tensor(source).to(device)
    source = source.reshape(-1, 1)
    src_mask = model.base.generate_square_subsequent_mask(source.size(0)).to(device)
    out = model(source, src_mask)
    next_token_ind =  int(torch.argmax(torch.nn.Softmax(dim=-1)(out[-1, 0, :])).detach()) #FORCE N=1
    return next_token_ind, out

def infer_next_tokens(sent, model, max_len=50):
    next_token_list = []
    next_token_index=-1
    iter = 0
    while next_token_index!=token2ind[EOS]:
        if iter>max_len:
            break
        next_token_index, out = infer_next_token(sent, model)
        next_token = ind2token.get(next_token_index)
        sent+= next_token
        next_token_list.append(next_token)
    return next_token_list


sent = "Bonjour les"
out = infer_next_tokens(sent, model_pretraining)
print(sent + " " + s.decode_pieces(out[:-1]))

### Supervised task

In [ ]:
data_loader_validation = get_loader(
    downstream_path_data_valid,
    downstream_path_labels_valid,
    token2ind=token2ind,
    # batch_size=20,
    batch_size=1, # Let's use a batch size of 1 so a sentence always has maximum length
    task='classification',
    shuffle=False # AVOID SHUFFLING FOR OBVIOUS REASONS - even if we're going through the whole validation set - let's do it in the same order everytime (batches will always be the same, no matter the configuration)
)
print(f"{len(data_loader_validation.dataset)} sentences in the validation set")

In [ ]:
for idx, sentence_batch in enumerate(data_loader_validation):
    if idx>10:
        break
    first_sentence = sentence_batch[0][:, 0]
    print(tensor_to_sentence(first_sentence))
    print(bool(sentence_batch[1][0]))

In [ ]:
# a function to evaluate the validation accuracy of the model.
def evaluate_accuracy(data_loader: DataLoader, model):
    accuracy = 0
    for _idx, sentence_batch in enumerate(data_loader):
        input_sentences = sentence_batch[0].to(device)
        target_labels = sentence_batch[1].to(device)
        logits = model(input_sentences, src_mask=None)[-1, ...]
        predicted_class = torch.argmax(logits)
        accuracy += torch.sum(predicted_class==target_labels).detach()
    accuracy = float(accuracy)/len(data_loader.dataset)
    return accuracy

def test_evaluate_accuracy():
    model_ds = Model(ntokens, nhead, nhid, nlayers, 2, dropout).to(device)
    model_ds.eval()
    acc = evaluate_accuracy(
        data_loader_validation,
        model_ds
    )
    print(f"accuracy of a randomly initalized classifier {acc} ~ shall be around 0.5")

# test_evaluate_accuracy()

In [ ]:
#save the base model to be loaded later in the fine-tuning phase
torch.save({"model_state_dict": model_pretraining.base.state_dict(),}, "pretrained_model_4layers_no_class_head.pt")

In [ ]:
from_scratch_settings = [False, True]

from_scratch_valid_acc = []
pretrained_valid_acc = []
lr = 0.0001

for from_scratch in from_scratch_settings:
    model_ds = Model(ntokens, nhead, nhid, nlayers, 2, dropout).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model_ds.parameters(), lr=lr)
    if not from_scratch:
        print("=====PRETRAINED MODEL======")
        #load checkpoint
        checkpoint = torch.load("pretrained_model_4layers_no_class_head.pt")
        #load state dict
        model_ds.base.load_state_dict(checkpoint['model_state_dict'])
    else:
        print("=====Trainig FROM SCRATCH======")
    epochs = 15
    # @TODO: check batch size subtlety on the last element (we want to classify the last feature of the sentence!)
    for epoch in range(1, epochs + 1):
        train(
            model_ds,
            downstream_path_data_train,
            downstream_path_labels_train,
            save_interval=-1,
            task=DS_TASK,
            freeze_backbone=not from_scratch,
            # batch_size=8,
            batch_size=1, # TO AVOID THE NEED TO RETRIEVE THE RIGHT LAST TOKEN IN A BATCH
            log_interval=50,
            epoch=epoch,
            name="_from_scratch" if from_scratch else "_pretrained"
        )
        acc = evaluate_accuracy(
            data_loader_validation,
            model_ds
        )
        if from_scratch:
            from_scratch_valid_acc.append(acc)
        else:
            pretrained_valid_acc.append(acc)
        print(f"{from_scratch=} - VALIDATION ACCURACY {acc}")

In [ ]:
#Visualize the accuracy
import matplotlib.pyplot as plt
plt.plot(100.*np.array(from_scratch_valid_acc), "-+r", label="training from scratch")
plt.plot(100.*np.array(pretrained_valid_acc), "-+b", label="pretrained model")
plt.grid()
plt.title("Classifier accuracy on a downstream task [Sentiment analyzis]")
plt.legend()
plt.ylim(0, 100)
plt.xlabel("Epoch")
plt.ylabel("Accuracy [%]")
plt.show()

In [ ]:
from_scratch_settings = [False, True]

from_scratch_valid_acc = []
pretrained_valid_acc = []
lr = 0.0001

for from_scratch in from_scratch_settings:
    model_ds = Model(ntokens, nhead, nhid, nlayers, 2, dropout).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model_ds.parameters(), lr=lr)
    if not from_scratch:
        print("=====PRETRAINED MODEL======")
        #load checkpoint
        checkpoint = torch.load("pretrained_model_4layers_no_class_head.pt")
        #load state dict
        model_ds.base.load_state_dict(checkpoint['model_state_dict'])
    else:
        print("=====Trainig FROM SCRATCH======")
    epochs = 15
    # @TODO: check batch size subtlety on the last element (we want to classify the last feature of the sentence!)
    for epoch in range(1, epochs + 1):
        train(
            model_ds,
            downstream_path_data_train,
            downstream_path_labels_train,
            save_interval=-1,
            task=DS_TASK,
            freeze_backbone=not from_scratch,
            batch_size=8,
            # batch_size=1, # TO AVOID THE NEED TO RETRIEVE THE RIGHT LAST TOKEN IN A BATCH
            log_interval=50,
            epoch=epoch,
            name="_from_scratch" if from_scratch else "_pretrained"
        )
        acc = evaluate_accuracy(
            data_loader_validation,
            model_ds
        )
        if from_scratch:
            from_scratch_valid_acc.append(acc)
        else:
            pretrained_valid_acc.append(acc)
        print(f"{from_scratch=} - VALIDATION ACCURACY {acc}")